In [6]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
from sliding_window_processor import collect_event_ids, FeatureExtractor, sequence_padder, windower
from collections import Counter

In [16]:
input_data = pd.read_csv("HDFS.log_structured.csv")
y = pd.read_csv("anomaly_label.csv")

In [17]:
input_data.head(25)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>
5,6,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>
6,7,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,e3df2680,Received block <*> of size <*> from <*>
7,8,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,e3df2680,Received block <*> of size <*> from <*>
8,9,81109,203519,147,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>
9,10,81109,203519,147,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,e3df2680,Received block <*> of size <*> from <*>


In [11]:
re_pat = r"(blk_-?\d+)"
col_names = ["BlockId", "EventSequence"]
events_df = collect_event_ids(input_data, re_pat, col_names) 

In [18]:
events_df = events_df.merge(y, on="BlockId")
events_df

,BlockId,EventSequence,Label_x,Label_y
0,blk_-1608999687919862906,"[09a53393, 3d91fa85, 09a53393, 09a53393, d38aa...",Normal,Normal
1,blk_7503483334202473044,"[09a53393, 09a53393, 3d91fa85, 09a53393, d38aa...",Normal,Normal
2,blk_-3544583377289625738,"[09a53393, 3d91fa85, 09a53393, 09a53393, d38aa...",Anomaly,Anomaly
3,blk_-9073992586687739851,"[09a53393, 3d91fa85, 09a53393, 09a53393, d38aa...",Normal,Normal
4,blk_7854771516489510256,"[09a53393, 09a53393, 3d91fa85, 09a53393, d38aa...",Normal,Normal
...,...,...,...,...
575056,blk_1019720114020043203,"[09a53393, 3d91fa85, 09a53393, 09a53393, 5d5de...",Normal,Normal
575057,blk_-2683116845478050414,"[09a53393, 3d91fa85, 09a53393, 09a53393, d38aa...",Normal,Normal
575058,blk_5595059397348477632,"[09a53393, 09a53393, 09a53393, 3d91fa85, d38aa...",Normal,Normal
575059,blk_1513937873877967730,"[09a53393, 09a53393, 09a53393, 3d91fa85, d38aa...",Normal,Normal
